

---

# **Chapter 33: Memory Management**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand Dart's generational garbage collection model and its implications
- Identify and prevent memory leaks in Flutter applications (streams, controllers, contexts)
- Implement proper disposal patterns for controllers, listeners, and animations
- Configure and optimize image caching with memory limits
- Handle large datasets efficiently without crashing the app
- Use DevTools Memory view to diagnose memory issues
- Implement weak references and cleanup patterns for long-lived objects

---

## **Prerequisites**

- Completed Chapter 32: Build Optimization (understanding of widget lifecycle)
- Completed Chapter 6: Asynchronous Programming (Streams, Futures)
- Understanding of StatefulWidget lifecycle (initState, dispose)
- Familiarity with image loading in Flutter (Image.network, AssetImage)

---

## **33.1 Dart's Garbage Collection**

Dart uses a generational garbage collection model optimized for UI applications. Understanding how it works helps you write memory-efficient code.

### **Generational GC Model**

```dart
// File: lib/memory/gc_concepts.dart

import 'package:flutter/material.dart';

class GarbageCollectionDemo {
  // Dart's heap is divided into two generations:
  
  // 1. NEW GENERATION (Young Space)
  // - Stores newly allocated objects
  // - Small, frequently collected (scavenge/young GC)
  // - Very fast (microseconds)
  // - Most objects die here (temporary widgets, local variables)
  
  void newGenerationExample() {
    // These objects start in new generation
    final tempList = [1, 2, 3]; // Allocated in new gen
    final tempString = 'Hello'; // Allocated in new gen
    
    // When method ends, these become unreachable
    // Next scavenge GC reclaims this memory
  }
  
  // 2. OLD GENERATION (Old Space)
  // - Stores long-lived objects that survived multiple new gen GCs
  // - Larger, collected less frequently (mark-sweep/old GC)
  // - Slower (milliseconds), causes UI jank if triggered during frame
  // - App state, cached images, singleton services live here
  
  static final _cache = <String, dynamic>{}; // Promoted to old gen
  
  void oldGenerationExample() {
    // Objects surviving ~10 new gen collections are promoted
    for (var i = 0; i < 1000; i++) {
      _cache['key$i'] = 'value$i'; // Survives, promoted to old gen
    }
    
    // These stay in memory until old gen GC runs
    // or app explicitly clears cache
  }
}

// Memory allocation patterns
class AllocationPatterns extends StatefulWidget {
  @override
  _AllocationPatternsState createState() => _AllocationPatternsState();
}

class _AllocationPatternsState extends State<AllocationPatterns> {
  // GOOD: Single allocation, survives in old gen
  final _controller = TextEditingController();
  
  // GOOD: Static const objects allocated once at compile time
  static const _padding = EdgeInsets.all(16.0);
  
  // BAD: Allocates new object on every build
  final _badPadding = EdgeInsets.all(16.0); // Instance field, but not const
  
  // WORSE: Creates new objects every build if defined in build()
  
  @override
  Widget build(BuildContext context) {
    // BAD: New object every build - pressure on new gen GC
    // final padding = EdgeInsets.all(16.0);
    
    // GOOD: Reuses const object
    // Uses _padding from class level or defines const locally
    
    return Padding(
      padding: _padding, // Reuses same instance
      child: TextField(controller: _controller),
    );
  }
  
  @override
  void dispose() {
    // CRITICAL: Release native resources immediately
    // GC won't collect immediately, but this frees native memory
    _controller.dispose();
    super.dispose();
  }
}
```

**Explanation:**

- **Generational hypothesis**: Most objects die young (temporary calculations, widget build results). Dart optimizes for this by having a small, fast new generation.
- **New Generation (Young Space)**: Uses "scavenge" collection—copies live objects to survivor space, drops dead ones. Very fast but frequent. If your app allocates many temporary objects, you'll see many scavenge GCs.
- **Old Generation (Old Space)**: Uses mark-sweep. When new gen survivors fill up, they're promoted here. Old gen GC is slow and stops the world (pauses UI thread). You want to minimize old gen GC during animations.
- **Promotion**: Objects surviving ~10 scavenge cycles are promoted to old gen. Long-lived objects (services, caches) should be created early so they promote quickly and stop being copied during new gen GC.

---

## **33.2 Memory Leaks in Flutter**

Memory leaks occur when objects are no longer needed but cannot be garbage collected because something still references them.

### **Common Leak Patterns**

```dart
// File: lib/memory/memory_leaks.dart
import 'package:flutter/material.dart';
import 'dart:async';

// LEAK PATTERN 1: Stream subscriptions without cancellation
class StreamLeakExample extends StatefulWidget {
  @override
  _StreamLeakExampleState createState() => _StreamLeakExampleState();
}

class _StreamLeakExampleState extends State<StreamLeakExample> {
  // LEAK: Subscription holds reference to State object
  // Even after widget is disposed, stream keeps State alive
  StreamSubscription? _subscription;
  
  @override
  void initState() {
    super.initState();
    
    // DANGEROUS: Listening to infinite stream
    _subscription = Stream.periodic(Duration(seconds: 1)).listen((_) {
      // This closure captures 'this' (State object)
      // As long as stream emits, State cannot be garbage collected
      setState(() {
        print('Updating state');
      });
    });
  }
  
  @override
  void dispose() {
    // FORGETTING THIS causes leak
    // _subscription?.cancel(); // Missing!
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    return Text('Leaking Stream');
  }
}

// FIXED VERSION
class FixedStreamExample extends StatefulWidget {
  @override
  _FixedStreamExampleState createState() => _FixedStreamExampleState();
}

class _FixedStreamExampleState extends State<FixedStreamExample> {
  StreamSubscription? _subscription;
  
  @override
  void initState() {
    super.initState();
    _subscription = Stream.periodic(Duration(seconds: 1)).listen((_) {
      if (mounted) { // Check widget still in tree
        setState(() {});
      }
    });
  }
  
  @override
  void dispose() {
    _subscription?.cancel(); // Release reference
    _subscription = null; // Clear reference
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) => Text('Fixed Stream');
}

// LEAK PATTERN 2: AnimationController not disposed
class AnimationLeak extends StatefulWidget {
  @override
  _AnimationLeakState createState() => _AnimationLeakState();
}

class _AnimationLeakState extends State<AnimationLeak>
    with SingleTickerProviderStateMixin {
  // LEAK: Ticker holds reference to State via TickerProvider
  late AnimationController _controller;
  
  @override
  void initState() {
    super.initState();
    _controller = AnimationController(
      vsync: this,
      duration: Duration(seconds: 1),
    )..repeat(); // Infinite animation keeps ticking
    
    // Even if widget is removed, controller keeps running
    // and holding reference to vsync (this State)
  }
  
  @override
  void dispose() {
    // MISSING: _controller.dispose();
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) => Container();
}

// LEAK PATTERN 3: Global singletons holding widget references
class SingletonLeak {
  // Static singleton lives for app lifetime (old gen)
  static final SingletonLeak _instance = SingletonLeak._internal();
  factory SingletonLeak() => _instance;
  SingletonLeak._internal();
  
  // LEAK: Holding reference to BuildContext or State
  BuildContext? _context;
  
  void registerContext(BuildContext context) {
    _context = context; // Widget cannot be garbage collected!
  }
  
  // FIX: Use WeakReference (Dart 2.17+)
  WeakReference<BuildContext>? _weakContext;
  
  void registerContextFixed(BuildContext context) {
    _weakContext = WeakReference(context);
  }
  
  void doSomething() {
    final context = _weakContext?.target;
    if (context != null) {
      // Context still alive, safe to use
    } else {
      // Context was garbage collected
    }
  }
}

// LEAK PATTERN 4: Closures capturing large objects
class ClosureLeak extends StatefulWidget {
  @override
  _ClosureLeakState createState() => _ClosureLeakState();
}

class _ClosureLeakState extends State<ClosureLeak> {
  late List<int> _largeData; // 1MB of data
  
  @override
  void initState() {
    super.initState();
    _largeData = List.generate(100000, (i) => i);
    
    // LEAK: Future captures 'this' via closure
    Future.delayed(Duration(minutes: 5), () {
      // Even if widget disposed immediately, this closure
      // keeps State alive for 5 minutes
      print(_largeData.length);
    });
  }
  
  // FIX: Use local variable and check mounted
  @override
  void initStateFixed() {
    super.initState();
    final largeData = List.generate(100000, (i) => i);
    
    Future.delayed(Duration(minutes: 5), () {
      // Check if widget still mounted
      if (mounted) {
        print(largeData.length);
      }
    });
  }
  
  @override
  Widget build(BuildContext context) => Container();
}

// LEAK PATTERN 5: ImageCache holding large images indefinitely
class ImageCacheLeak extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: 1000,
      itemBuilder: (context, index) {
        // By default, Flutter caches all images in memory
        // Scrolling through 1000 large images fills memory
        return Image.network(
          'https://example.com/large_image_$index.jpg',
          // No cache width/height specified = full resolution in memory
        );
      },
    );
  }
}
```

**Explanation:**

- **Stream subscriptions**: The closure passed to `listen()` implicitly captures `this`. If you don't cancel the subscription in `dispose()`, the stream holds your State object alive forever.
- **AnimationController**: The `vsync` parameter creates a reference from the ticker to the State. `dispose()` must be called to break this reference.
- **Global references**: Singletons and static variables live for the app lifetime. Holding references to widgets/contexts prevents their entire subtrees from being collected. Use `WeakReference` (Dart 2.17+) when you need to reference widgets without preventing collection.
- **Closures in async operations**: Future closures capture the enclosing scope. If they reference `this` or instance fields, they keep the State alive until the Future completes.
- **ImageCache**: Flutter's default `ImageCache` keeps images in memory indefinitely (subject to size limits). Loading many large images without size constraints causes OOM crashes.

---

## **33.3 Proper Disposal Patterns**

Systematic resource cleanup prevents memory leaks and native resource exhaustion.

### **The Dispose Checklist**

```dart
// File: lib/memory/disposal_patterns.dart
import 'package:flutter/material.dart';
import 'dart:async';

// COMPREHENSIVE DISPOSAL EXAMPLE
class ProperDisposalWidget extends StatefulWidget {
  @override
  _ProperDisposalWidgetState createState() => _ProperDisposalWidgetState();
}

class _ProperDisposalWidgetState extends State<ProperDisposalWidget>
    with SingleTickerProviderStateMixin {
  
  // 1. TextEditingController - holds native text input resources
  late final TextEditingController _textController;
  
  // 2. FocusNode - holds native focus resources
  late final FocusNode _focusNode;
  
  // 3. ScrollController - holds scroll position and listeners
  late final ScrollController _scrollController;
  
  // 4. AnimationController - holds ticker and native animation resources
  late final AnimationController _animationController;
  
  // 5. StreamSubscriptions - hold references to stream and callbacks
  StreamSubscription? _dataSubscription;
  StreamSubscription? _eventSubscription;
  
  // 6. Timers - hold callbacks and scheduled tasks
  Timer? _debounceTimer;
  Timer? _pollingTimer;
  
  // 7. PageController, TabController, etc.
  late final PageController _pageController;
  
  @override
  void initState() {
    super.initState();
    
    // Initialize controllers
    _textController = TextEditingController();
    _focusNode = FocusNode();
    _scrollController = ScrollController();
    _pageController = PageController();
    
    _animationController = AnimationController(
      vsync: this,
      duration: Duration(milliseconds: 300),
    );
    
    // Setup listeners (must be removed in dispose)
    _textController.addListener(_onTextChanged);
    _scrollController.addListener(_onScroll);
    _focusNode.addListener(_onFocusChange);
    
    // Subscribe to streams
    _dataSubscription = _fetchDataStream().listen(_onData);
    _eventSubscription = _eventBus.stream.listen(_onEvent);
    
    // Start timers
    _pollingTimer = Timer.periodic(Duration(seconds: 30), (_) {
      _pollServer();
    });
  }
  
  void _onTextChanged() {
    // Debounce pattern - cancel previous timer
    _debounceTimer?.cancel();
    _debounceTimer = Timer(Duration(milliseconds: 500), () {
      _search(_textController.text);
    });
  }
  
  @override
  void dispose() {
    // CRITICAL: Cancel subscriptions first to prevent callbacks during disposal
    _dataSubscription?.cancel();
    _eventSubscription?.cancel();
    _dataSubscription = null;
    _eventSubscription = null;
    
    // Cancel timers
    _debounceTimer?.cancel();
    _pollingTimer?.cancel();
    _debounceTimer = null;
    _pollingTimer = null;
    
    // Remove listeners before disposing controllers
    _textController.removeListener(_onTextChanged);
    _scrollController.removeListener(_onScroll);
    _focusNode.removeListener(_onFocusChange);
    
    // Dispose controllers in reverse order of initialization
    _pageController.dispose();
    _animationController.dispose();
    _scrollController.dispose();
    _focusNode.dispose();
    _textController.dispose();
    
    super.dispose();
  }
  
  Stream<String> _fetchDataStream() async* {
    yield 'data';
  }
  
  void _onData(String data) {}
  void _onEvent(dynamic event) {}
  void _onScroll() {}
  void _onFocusChange() {}
  void _search(String query) {}
  void _pollServer() {}
  
  @override
  Widget build(BuildContext context) {
    return Container();
  }
}

// ABSTRACTION: Disposable mixin for complex scenarios
mixin Disposable on StatefulWidget {
  void disposeResources();
}

// Alternative: Use flutter_hooks or riverpod for automatic disposal
// These packages manage lifecycle automatically
```

**Explanation:**

- **Order matters**: Cancel subscriptions and timers first to prevent callbacks during disposal. Then remove listeners. Finally dispose controllers. If you dispose a controller before removing listeners, some implementations throw exceptions.
- **Nullify references**: Set controllers/subscriptions to null after disposal. This helps catch use-after-dispose bugs (null pointer exceptions instead of silent failures).
- **Debouncing**: Timer-based operations like search debouncing must cancel pending timers in `dispose()`, or the callback may run after the widget is gone (causing "setState called after dispose" errors).
- **Mixins**: For widgets with many resources, consider a mixin or helper class to standardize disposal patterns and prevent mistakes.

---

## **33.4 Image Caching and Memory Limits**

Images are the #1 cause of memory issues in Flutter apps. Proper configuration prevents OOM crashes.

### **Image Memory Management**

```dart
// File: lib/memory/image_optimization.dart
import 'package:flutter/material.dart';

class ImageMemoryManagement extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: 1000,
      itemBuilder: (context, index) {
        return OptimizedImageItem(index: index);
      },
    );
  }
}

class OptimizedImageItem extends StatelessWidget {
  final int index;
  
  const OptimizedImageItem({required this.index});
  
  @override
  Widget build(BuildContext context) {
    return Image.network(
      'https://example.com/image_$index.jpg',
      
      // CRITICAL: Specify cache dimensions
      // Resizes image to this resolution before caching
      // Without this, full 4K images are stored in memory
      cacheWidth: 600,
      cacheHeight: 400,
      
      // Fit mode determines how image fills space
      fit: BoxFit.cover,
      
      // Placeholder while loading (prevents layout jumps)
      loadingBuilder: (context, child, loadingProgress) {
        if (loadingProgress == null) return child;
        return Container(
          height: 200,
          color: Colors.grey[300],
          child: Center(
            child: CircularProgressIndicator(
              value: loadingProgress.expectedTotalBytes != null
                  ? loadingProgress.cumulativeBytesLoaded /
                    loadingProgress.expectedTotalBytes!
                  : null,
            ),
          ),
        );
      },
      
      // Error fallback
      errorBuilder: (context, error, stackTrace) {
        return Container(
          height: 200,
          color: Colors.red[100],
          child: Icon(Icons.error),
        );
      },
    );
  }
}

// Custom cache configuration
class CustomImageCache extends WidgetsFlutterBinding {
  @override
  ImageCache createImageCache() {
    final cache = super.createImageCache();
    
    // Set maximum image cache size (default is 100MB)
    // Lower this on low-memory devices
    cache.maximumSizeBytes = 50 * 1024 * 1024; // 50MB
    
    // Maximum number of images (default 1000)
    cache.maximumSize = 100;
    
    return cache;
  }
}

// To use custom cache:
// void main() {
//   CustomImageCache();
//   runApp(MyApp());
// }

// Clearing image cache manually
class CacheManagement {
  static void clearImageCache() {
    // Clear all cached images
    PaintingBinding.instance.imageCache.clear();
    
    // Clear only live images (retained by widgets)
    PaintingBinding.instance.imageCache.clearLiveImages();
  }
  
  static void evictSpecificImage(String url) {
    // Remove specific image from cache
    final provider = NetworkImage(url);
    PaintingBinding.instance.imageCache.evict(provider);
  }
}

// Memory-conscious image preloading
class PreloadManager {
  static final _preloadQueue = <String>[];
  
  static void preloadImages(BuildContext context, List<String> urls) {
    for (final url in urls.take(5)) { // Only preload first 5
      precacheImage(
        NetworkImage(url),
        context,
        onError: (exception, stackTrace) {
          print('Failed to preload $url: $exception');
        },
      );
    }
  }
}

// Handling large local images
class LocalImageOptimization extends StatelessWidget {
  final String assetPath;
  
  const LocalImageOptimization({required this.assetPath});
  
  @override
  Widget build(BuildContext context) {
    // For large local assets, use ResizeImage
    return Image(
      image: ResizeImage(
        AssetImage(assetPath),
        width: 800, // Decode at reduced resolution
        allowUpscaling: false,
      ),
    );
  }
}
```

**Explanation:**

- **cacheWidth/cacheHeight**: These are the most important parameters for memory. They tell Flutter to resize the image to these dimensions before storing in memory. A 4000x3000 image displayed at 400x300 still consumes 48MB if not resized (4000 * 3000 * 4 bytes). With `cacheWidth: 800`, it uses only 1.9MB.
- **ImageCache limits**: Default cache is 100MB. On low-end devices with 2GB RAM, this is too aggressive. Override `createImageCache()` to set appropriate limits.
- **Live images vs cached**: `clear()` removes decoded images but keeps "live" images (currently displayed). `clearLiveImages()` removes everything, causing visible images to reload.
- **Preloading**: `precacheImage()` loads images before they're needed (e.g., next page in carousel), but only preload a few images to avoid memory spikes.
- **ResizeImage**: For large local assets bundled in the app, `ResizeImage` decodes at reduced resolution, saving memory.

---

## **33.5 Handling Large Datasets**

Displaying thousands of items requires pagination, efficient data structures, and windowing.

### **Efficient Large List Handling**

```dart
// File: lib/memory/large_datasets.dart
import 'package:flutter/material.dart';
import 'package:infinite_scroll_pagination/infinite_scroll_pagination.dart';

// PATTERN 1: Pagination with PagingController
class PaginatedList extends StatefulWidget {
  @override
  _PaginatedListState createState() => _PaginatedListState();
}

class _PaginatedListState extends State<PaginatedList> {
  // PagingController manages page requests and state
  final PagingController<int, Item> _pagingController = 
      PagingController(firstPageKey: 1);
  
  @override
  void initState() {
    super.initState();
    _pagingController.addPageRequestListener((pageKey) {
      _fetchPage(pageKey);
    });
  }
  
  Future<void> _fetchPage(int pageKey) async {
    try {
      // Fetch 20 items at a time
      final newItems = await fetchItems(page: pageKey, limit: 20);
      
      final isLastPage = newItems.length < 20;
      if (isLastPage) {
        _pagingController.appendLastPage(newItems);
      } else {
        final nextPageKey = pageKey + 1;
        _pagingController.appendPage(newItems, nextPageKey);
      }
    } catch (error) {
      _pagingController.error = error;
    }
  }
  
  @override
  Widget build(BuildContext context) {
    return PagedListView<int, Item>(
      pagingController: _pagingController,
      builderDelegate: PagedChildBuilderDelegate<Item>(
        itemBuilder: (context, item, index) => ItemTile(item: item),
        // Only keeps visible items + cacheExtent in memory
      ),
    );
  }
  
  @override
  void dispose() {
    _pagingController.dispose();
    super.dispose();
  }
}

// PATTERN 2: Windowing/Viewport optimization
class WindowedList extends StatelessWidget {
  final List<String> items = List.generate(10000, (i) => 'Item $i');
  
  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: items.length,
      
      // Add itemExtent for massive performance gain
      // Flutter knows exact height without measuring
      itemExtent: 80.0,
      
      // CacheExtent controls how many items are built outside viewport
      // Default is 250 logical pixels
      // Reduce for lower memory usage, increase for smoother scrolling
      cacheExtent: 100.0,
      
      itemBuilder: (context, index) {
        // Only builds items currently visible + cacheExtent
        return ListTile(
          title: Text(items[index]),
        );
      },
    );
  }
}

// PATTERN 3: Heavy object virtualization
class VirtualizedHeavyItems extends StatefulWidget {
  @override
  _VirtualizedHeavyItemsState createState() => _VirtualizedHeavyItemsState();
}

class _VirtualizedHeavyItemsState extends State<VirtualizedHeavyItems> {
  // Only store lightweight metadata in memory
  final List<ItemMetadata> _metadata = [];
  
  // Heavy data loaded on-demand and cached with LRU policy
  final _heavyDataCache = <int, HeavyItemData>{};
  static const _maxCacheSize = 50; // Only keep 50 heavy items in memory
  
  @override
  void initState() {
    super.initState();
    // Load only metadata (lightweight)
    _loadMetadata();
  }
  
  void _loadMetadata() async {
    // Load just IDs and titles
    final metadata = await fetchMetadataOnly();
    setState(() {
      _metadata.addAll(metadata);
    });
  }
  
  Future<HeavyItemData> _getHeavyData(int id) async {
    // Check memory cache first
    if (_heavyDataCache.containsKey(id)) {
      return _heavyDataCache[id]!;
    }
    
    // Evict oldest if cache full
    if (_heavyDataCache.length >= _maxCacheSize) {
      final oldestKey = _heavyDataCache.keys.first;
      _heavyDataCache.remove(oldestKey);
    }
    
    // Load from disk/network
    final data = await fetchHeavyData(id);
    _heavyDataCache[id] = data;
    return data;
  }
  
  @override
  Widget build(BuildContext context) {
    return ListView.builder(
      itemCount: _metadata.length,
      itemBuilder: (context, index) {
        final meta = _metadata[index];
        
        return FutureBuilder<HeavyItemData>(
          future: _getHeavyData(meta.id),
          builder: (context, snapshot) {
            if (snapshot.hasData) {
              return HeavyItemTile(data: snapshot.data!);
            }
            return ListTile(title: Text('Loading ${meta.title}...'));
          },
        );
      },
    );
  }
}

// Data classes
class Item {
  final String id;
  final String title;
  Item(this.id, this.title);
}

class ItemMetadata {
  final int id;
  final String title;
  ItemMetadata(this.id, this.title);
}

class HeavyItemData {
  final int id;
  final String content;
  final List<int> binaryData; // Large data
  
  HeavyItemData(this.id, this.content, this.binaryData);
}

class ItemTile extends StatelessWidget {
  final Item item;
  const ItemTile({required this.item});
  
  @override
  Widget build(BuildContext context) => ListTile(title: Text(item.title));
}

class HeavyItemTile extends StatelessWidget {
  final HeavyItemData data;
  const HeavyItemTile({required this.data});
  
  @override
  Widget build(BuildContext context) => ListTile(
    title: Text(data.title),
    subtitle: Text('${data.binaryData.length} bytes'),
  );
}

// Mock functions
Future<List<Item>> fetchItems({required int page, required int limit}) async {
  await Future.delayed(Duration(milliseconds: 500));
  return List.generate(limit, (i) => Item('${page}_$i', 'Item ${page}_$i'));
}

Future<List<ItemMetadata>> fetchMetadataOnly() async {
  return List.generate(1000, (i) => ItemMetadata(i, 'Item $i'));
}

Future<HeavyItemData> fetchHeavyData(int id) async {
  await Future.delayed(Duration(milliseconds: 100));
  return HeavyItemData(id, 'Content $id', List.generate(10000, (i) => i));
}

// Using sqflite for local large datasets
// Store in SQLite, query with pagination, keep only current page in memory
```

**Explanation:**

- **Pagination**: Don't load 10,000 items at once. Use `infinite_scroll_pagination` or similar packages to load 20-50 items at a time as the user scrolls.
- **Metadata vs Data**: Keep lightweight metadata (ID, title, thumbnail URL) in memory for the full list, but load heavy data (full content, large images) only when needed.
- **LRU Cache**: For on-demand loaded data, implement a Least Recently Used cache with a fixed size (e.g., 50 items). When cache is full, remove the oldest item before adding new.
- **itemExtent**: Critical for large lists. Without it, Flutter must measure every item to know the scroll position, causing O(n) layout cost.
- **Database**: For truly massive datasets (100k+ rows), use SQLite via `sqflite`. Query with `LIMIT` and `OFFSET`, only keeping the current page in widget state.

---

## **Chapter Summary**

In this chapter, we covered strategies to manage memory efficiently and prevent leaks in Flutter applications:

### **Key Takeaways:**

1. **Garbage Collection**: Dart uses generational GC. New generation (young space) is fast and frequent; old generation is slow and causes jank. Minimize allocations during animations, and promote long-lived objects early.

2. **Memory Leaks**: Common causes include uncanceled stream subscriptions, undisposed AnimationControllers, global singletons holding widget references, and closures capturing State objects. Always cancel/dispose in `dispose()`.

3. **Disposal Checklist**: Cancel subscriptions and timers first, remove listeners, then dispose controllers. Use `WeakReference` (Dart 2.17+) when global objects need to reference widgets without preventing GC.

4. **Image Optimization**: Always specify `cacheWidth` and `cacheHeight` to resize images before caching. Configure `ImageCache` size limits for low-memory devices. Clear cache when navigating away from image-heavy screens.

5. **Large Datasets**: Implement pagination with `PagingController`. Use `ListView.builder` with `itemExtent` for massive lists. Keep only metadata in memory, load heavy data on-demand with LRU caching.

6. **DevTools Memory View**: Use the Memory tab to track heap usage, identify retained objects, and detect leaks by taking heap snapshots before/after navigation.

### **Next Steps:**

Chapter 34 will cover **Rendering Optimization**:
- Custom painters and canvas optimization
- Shader compilation and warm-up
- Impeller rendering engine (new)
- Reducing overdraw and opacity layers
- Slivers and custom scroll physics

---

**End of Chapter 33**

---

# **Next Chapter: Chapter 34 - Rendering Optimization**

Chapter 34 will dive into the Paint phase optimizations, custom rendering, and leveraging Flutter's Impeller engine for smooth graphics performance.

